# Assignment 2 For LightBeam.ai
-   Design a suitable Data Structure for File System

### Install Required Packages

In [8]:
import json

In [10]:
def load_config(config_file):
    """
    This function loads a JSON configuration file and returns the corresponding data structure.
    Args:
        config_file (str): The path to the JSON configuration file.
    Returns:
        dict: A dictionary representing the configuration data structure.
    """
    with open(config_file) as f:
        return json.load(f)

### Add Folder Function

In [11]:
class FolderAlreadyExistsError(Exception):
    pass
class PathNotFoundError(Exception):
    pass

In [12]:
def add_folder(tree, path, folder_id, name):
    """
    This function adds a new folder to the directory tree at the specified path.

    Args:
        tree (dict): The root node of the directory tree.
        path (list): A list of folder names representing the path to the parent folder of the new folder.
        folder_id (int): The unique identifier for the new folder.
        name (str): The name of the new folder.

    Raises:
        PathNotFoundError: If the specified path does not exist in the directory tree.
        FolderAlreadyExistsError: If a folder with the specified name already exists at the specified path.

    Returns:
        None
    """
    parent = tree
    for folder in path:
        parent = next((subfolder for subfolder in parent['subfolders'] if subfolder['name'] == folder), None)
        if parent is None:
            # raise ValueError('Path not found')
            raise PathNotFoundError(f"The path {path} does not exists. Enter the valid path")
    if next((subfolder for subfolder in parent['subfolders'] if subfolder['name'] == name), None) is not None:
        raise FolderAlreadyExistsError(f'Folder with name "{name}" already exists at path "{"/".join(path)}"')
    new_folder = {'id': folder_id, 'name': name, 'subfolders': []}
    parent['subfolders'].append(new_folder)
    print(json.dumps(tree))


#### Call Add_Folder Function

In [15]:
# To test the above function
tree = load_config('config.json')
add_folder(tree, path=['folder1'], folder_id=3, name='new_folder')

{"id": "root", "name": "Root", "subfolders": [{"id": 1, "name": "folder1", "subfolders": [{"id": 2, "name": "folder2", "subfolders": []}, {"id": 3, "name": "folder3", "subfolders": []}, {"id": 3, "name": "new_folder", "subfolders": []}]}]}


### Remove Folder Function

In [16]:
class FolderNotFoundError(Exception):
    pass

In [17]:
def remove_folder(tree, path, name):
    """
    This function removes a folder from the directory tree at the specified path.

    Args:
        tree (dict): The root node of the directory tree.
        path (list): A list of folder names representing the path to the parent folder of the new folder.
        folder_id (int): The unique identifier for the new folder.
        name (str): The name of the new folder.

    Raises:
        FolderNotFoundError: If a folder with the specified name does not exists at the specified path.

    Returns:
        None
    """
    parent = tree
    for folder in path:
        parent = next((dict_subfolder for dict_subfolder in parent['subfolders'] if
                       dict_subfolder['name'] ==
                       folder), None)
        if parent is None:
            raise FolderNotFoundError(f'The Folder {folder} not found')
    parent['subfolders'] = [subfolder for subfolder in parent['subfolders'] if subfolder['name'] != name]

    print(f"Updated tree structure is \n {tree}")

#### Call Remove Folder Function

In [18]:
# To test the above function
tree = load_config('config.json')
print(f"Current tree structure is \n {tree}")
print("*" * 80)
remove_folder(tree, ['folder1'], 'folder2')

Current tree structure is 
 {'id': 'root', 'name': 'Root', 'subfolders': [{'id': 1, 'name': 'folder1', 'subfolders': [{'id': 2, 'name': 'folder2', 'subfolders': []}, {'id': 3, 'name': 'folder3', 'subfolders': []}]}]}
********************************************************************************
Updated tree structure is 
 {'id': 'root', 'name': 'Root', 'subfolders': [{'id': 1, 'name': 'folder1', 'subfolders': [{'id': 3, 'name': 'folder3', 'subfolders': []}]}]}


### Fetch Folder path Function

In [19]:
class FolderIdNotFoundError(Exception):
    pass

In [21]:
def fetch_folder_path(tree, folder_id):
    """
    This function return a path from the directory tree of the specified folder.

    Args:
        tree (dict): The root node of the directory tree.
        folder_id (int): The unique identifier for the new folder.

    Raises:
        FolderNotFoundError: If a folder with the specified name does not exists at the specified path.

    Returns:
        path (list)
    """
    def _traverse(node, path):
        if node['id'] == folder_id:
            return [tree['name']] + path
        for subfolder in node['subfolders']:
            result = _traverse(subfolder, path + [subfolder['name']])
            if result:
                return result
        return None

    path = _traverse(tree, [])

    if path is None:
        raise FolderIdNotFoundError(f'Folder ID {folder_id} not found')
    return path

#### Call fetch path function

In [25]:
# To test the above function
tree = load_config('config.json')
path = fetch_folder_path(tree, 2)
print("*" * 80)
print(f" The path is \n {'/'.join(path)}")
print("*" * 80)

********************************************************************************
 The path is 
 Root/folder1/folder2
********************************************************************************


### Update_folder_name

In [26]:
def update_folder_name(tree, folder_id, new_name):
    """
        This function updates the name of the specified folder in a directory tree.
        Args:
            tree (dict): The root node of the directory tree.
            folder_id (int): The unique identifier for the new folder.
            new_name (str): The new name of the folder.
        Raises:
            FolderNotFoundError: If a folder with the specified name does not exists at the specified path.
        Returns:
            None
        Functionality:
            The function first calls the fetch_folder_path() function to get the path of the folder with the specified ID. If the folder is not found, it raises a FolderNotFoundError. Otherwise, it recursively traverses the directory tree to find the specified folder and updates its name. The updated tree structure is then printed.
    """

    path = fetch_folder_path(tree, folder_id)
    if path is None:
        raise FolderNotFoundError('Folder not found')

    def _traverse(node):
        if node['id'] == folder_id:
            node['name'] = new_name
            return
        for subfolder in node['subfolders']:
            _traverse(subfolder)

    _traverse(tree)
    print("*" * 80)
    print(f'Folder name updated from {path[-1]} to {new_name}')
    print("*" * 80)
    print(f"Updated tree structure is \n {tree}")
    print("*" * 80)


In [27]:
tree = load_config('config.json')
print(f"Current tree structure is \n {tree}")
update_folder_name(tree, 1, 'new_name_folder1')


Current tree structure is 
 {'id': 'root', 'name': 'Root', 'subfolders': [{'id': 1, 'name': 'folder1', 'subfolders': [{'id': 2, 'name': 'folder2', 'subfolders': []}, {'id': 3, 'name': 'folder3', 'subfolders': []}]}]}
********************************************************************************
Folder name updated from folder1 to new_name_folder1
********************************************************************************
Updated tree structure is 
 {'id': 'root', 'name': 'Root', 'subfolders': [{'id': 1, 'name': 'new_name_folder1', 'subfolders': [{'id': 2, 'name': 'folder2', 'subfolders': []}, {'id': 3, 'name': 'folder3', 'subfolders': []}]}]}
********************************************************************************
